# Motor Vehicle Collisions - Crash Analysis, New York City
In this notebook I have analyzed the motor vehicle collisions in New York City, to understand various parameters and reasons for these crash and devise possible solution to mitigate the problem. This notebooks provides a detailed description and understanding of the dataset, highlighting few key variables and statistics that will be instrumental in devising solutions. The data has been sourced from the following link: 
 https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95.

## 1.Importing required libraries:
Here we import all the required libraries to facilitate data analysis and prevent any errors in scripting code relating to importing libraries.

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os
import scipy
import math

##### Read the file into a dataframe using ```read_pickle```

In [4]:
df = pd.read_pickle(r"shared/Motor_Vehicle_Collisions_-_Crashes.pkl")
df

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


As we can see from the dataframe the ```shape``` of the dataframe is ```(2018245,28)```. There are over 2 million+ records with 28 columns/features. With the help of these features we can understand the dataset better and determine the KPIs driving the crashes is NYC.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018245 entries, 0 to 2018244
Data columns (total 28 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   CRASH DATE_CRASH TIME          datetime64[ns]
 1   BOROUGH                        object        
 2   ZIP CODE                       object        
 3   LATITUDE                       float64       
 4   LONGITUDE                      float64       
 5   LOCATION                       object        
 6   ON STREET NAME                 object        
 7   CROSS STREET NAME              object        
 8   OFF STREET NAME                object        
 9   NUMBER OF PERSONS INJURED      float64       
 10  NUMBER OF PERSONS KILLED       float64       
 11  NUMBER OF PEDESTRIANS INJURED  int64         
 12  NUMBER OF PEDESTRIANS KILLED   int64         
 13  NUMBER OF CYCLIST INJURED      int64         
 14  NUMBER OF CYCLIST KILLED       int64         
 15  NUMBER OF MOTOR

To help future calculation create new columns extracting the year and month from the ```CRASH DATE_CRASH TIME``` column using the ```dt.year``` and ```dt.month``` attributes of datetime.

In [6]:
df['CRASH_YEAR'] = df['CRASH DATE_CRASH TIME'].dt.year
df['CRASH_MONTH'] = df['CRASH DATE_CRASH TIME'].dt.month

In [7]:
df.head(5)

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,CRASH_YEAR,CRASH_MONTH
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN,2021,9
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN,2022,3
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN,2022,6
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN,2021,9
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN,2021,12


## Question 1 - Which borough has had the second highest total number of crashes reported since 2012?
Determined the crash count of each NYC Borough, using the ```df.query()```,```groupby()```,```agg()```and ```sort_values()``` methods.

In [8]:
df_crashes_2012 = df.query('CRASH_YEAR >= 2012')
df_crashes_2012

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,CRASH_YEAR,CRASH_MONTH
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN,2021,9
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN,2022,3
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN,2022,6
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN,2021,9
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN,2021,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN,2023,7
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN,2023,7
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN,2023,7
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN,2023,7


In [9]:
df_crashes_2012_boroughs = df_crashes_2012.groupby('BOROUGH').agg({'COLLISION_ID': 'count'}).sort_values(by = 'COLLISION_ID', ascending = False)
print(df_crashes_2012_boroughs)

               COLLISION_ID
BOROUGH                    
BROOKLYN             441026
QUEENS               372457
MANHATTAN            313266
BRONX                205345
STATEN ISLAND         58297


## Question 2 - Which borough has the most crashes for every 100,000 people?
Divided the total count of crashes by the population, and multiply by 100,000 to get the crashes for every 100,000 people.

In [11]:
print((df_crashes_2012_boroughs.loc[:, 'COLLISION_ID']/[2648452, 2330295, 1638281, 1446788, 487155])*100000)

BOROUGH
BROOKLYN         16652.217975
QUEENS           15983.255339
MANHATTAN        19121.628097
BRONX            14193.164444
STATEN ISLAND    11966.827806
Name: COLLISION_ID, dtype: float64


## Question 3 - what is the leading cause of crashes using ```CONTRIBUTING FACTOR VEHICLE 1```?
Here the causes of crashes are identified and the max values of the causes are determined to facilitate further calculations.

In [12]:
df['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

CONTRIBUTING FACTOR VEHICLE 1
Unspecified                       692736
Driver Inattention/Distraction    401262
Failure to Yield Right-of-Way     119166
Following Too Closely             107467
Backing Unsafely                   75042
                                   ...  
Cell Phone (hand-held)                79
Windshield Inadequate                 77
Texting                               50
Listening/Using Headphones            24
1                                     10
Name: count, Length: 61, dtype: int64

In [13]:
df_coutributing_1 = df[['CONTRIBUTING FACTOR VEHICLE 1']]

In [14]:
values_to_exclude = ['Unspecified']
df_coutributing_1 = pd.DataFrame(df_coutributing_1[~df_coutributing_1['CONTRIBUTING FACTOR VEHICLE 1'].isin(values_to_exclude)])

In [15]:
df_contributing_1 = df_coutributing_1.groupby('CONTRIBUTING FACTOR VEHICLE 1').agg({'CONTRIBUTING FACTOR VEHICLE 1':'count'}).rename(columns = {'CONTRIBUTING FACTOR VEHICLE 1':'COUNT'}).sort_values(by='COUNT', ascending = False).reset_index()

In [16]:
df_contributing_1['PROPORTION'] = (df_contributing_1['COUNT']/df_contributing_1['COUNT'].sum())*100

In [17]:
df_contributing_1 

,CONTRIBUTING FACTOR VEHICLE 1,COUNT,PROPORTION
0,Driver Inattention/Distraction,401262,30.417970
1,Failure to Yield Right-of-Way,119166,9.033469
2,Following Too Closely,107467,8.146617
3,Backing Unsafely,75042,5.688616
4,Other Vehicular,62688,4.752111
5,Passing or Lane Usage Improper,55445,4.203050
6,Turning Improperly,49908,3.783314
7,Passing Too Closely,49848,3.778765
8,Fatigued/Drowsy,47343,3.588872
9,Unsafe Lane Changing,39711,3.010322


## Question 4 - The top 5 causes of crashes (ignoring 'Unspecified') account for what proportion of total crashes?

In [18]:
print(df_contributing_1['PROPORTION'].head().sum())

58.038783742090615


## Question 5 - what is the total count of accidents that involved two or more fatalities? 
Here a new dataframe is created to help in calculation the number accidents that resulted in fatalities. 

In [19]:
df_fatalities = df[['NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST KILLED']]
df_fatalities.sort_values(by = 'NUMBER OF PERSONS KILLED', ascending = False)

,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST KILLED
781939,8.0,6,2,0
1909664,5.0,0,0,5
818337,4.0,2,0,2
270897,4.0,0,0,4
1585460,4.0,0,0,4
...,...,...,...,...
949202,NaN,0,0,0
951826,NaN,0,0,0
957470,NaN,0,0,0
963358,NaN,0,0,0


In [20]:
df_fatalities.rename(columns = {'NUMBER OF PERSONS KILLED':'Number_Of_Persons_Killed'}, inplace = True)

/tmp/ipykernel_1124/3096320851.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fatalities.rename(columns = {'NUMBER OF PERSONS KILLED':'Number_Of_Persons_Killed'}, inplace = True)


In [21]:
df_fatalities_2_or_more = df_fatalities.query('Number_Of_Persons_Killed >= 2.0').sort_values(by = 'Number_Of_Persons_Killed', ascending = True)

In [22]:
df_fatalities_2_or_more

,Number_Of_Persons_Killed,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST KILLED
4137,2.0,0,0,2
1459694,2.0,0,0,2
1449089,2.0,0,0,2
1379399,2.0,2,0,0
1378036,2.0,0,0,2
...,...,...,...,...
1585460,4.0,0,0,4
270897,4.0,0,0,4
818337,4.0,2,0,2
1909664,5.0,0,0,5


Here we can see that the total number of records after the filtering the dataframe is 88. This is the number of accidents that resulted in 2 or more fatalities.

## Question 6 - On average, for every 1000 accidents, how many have resulted in at least one person dead?

In [23]:
df_fatalities_1_or_more = df_fatalities.query('Number_Of_Persons_Killed >= 1').sort_values('Number_Of_Persons_Killed', ascending = True)
df_fatalities_1_or_more

,Number_Of_Persons_Killed,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST KILLED
39,1.0,1,0,0
1388028,1.0,1,0,0
1390073,1.0,1,0,0
1390228,1.0,0,0,1
1390589,1.0,1,0,0
...,...,...,...,...
818337,4.0,2,0,2
270897,4.0,0,0,4
1585460,4.0,0,0,4
1909664,5.0,0,0,5


In [24]:
print((2804/2018245)*1000)

1.3893258747079764


## Question 7 - What proportion of accidents in the data do not have a ```Cross Street Name```?

In [25]:
df_NaN = pd.DataFrame(df.isna().sum(axis=0)).reset_index()

In [26]:
df_NaN.rename(columns = {0:'NaN_count', 'index':'Columns'}, inplace = True)

In [27]:
df_NaN.head()

,Columns,NaN_count
0,CRASH DATE_CRASH TIME,0
1,BOROUGH,627854
2,ZIP CODE,628092
3,LATITUDE,229685
4,LONGITUDE,229685


In [29]:
df_NaN.query('Columns == "CROSS STREET NAME"')

,Columns,NaN_count
7,CROSS STREET NAME,755532


In [30]:
print((755532/2018245))

0.37435098315615795


## Question 8 - The fields ```VEHICLE TYPE CODE 1``` and ```VEHICLE TYPE CODE 2``` represent the first two vehicles involved in the accident. Which combination of vehicles have the most number of accidents?

In [31]:
df_Vehicle_code = df[['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']]
df_Vehicle_code

,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2
0,Sedan,Sedan
1,Sedan,NaN
2,Sedan,Pick-up Truck
3,Sedan,NaN
4,NaN,NaN
...,...,...
2018240,Sedan,Sedan
2018241,Sedan,NaN
2018242,Taxi,NaN
2018243,Station Wagon/Sport Utility Vehicle,E-Bike


In [32]:
df_Vehicle_code.dropna(axis = 0)

,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2
0,Sedan,Sedan
2,Sedan,Pick-up Truck
5,Dump,Sedan
6,Sedan,Tractor Truck Diesel
7,Sedan,Sedan
...,...,...
2018235,Bike,Sedan
2018236,Ambulance,Moped
2018238,Station Wagon/Sport Utility Vehicle,Sedan
2018240,Sedan,Sedan


In [33]:
df_Combination = pd.DataFrame(df_Vehicle_code.value_counts()).reset_index()

In [34]:
df_Combination

,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,count
0,Sedan,Sedan,197944
1,PASSENGER VEHICLE,PASSENGER VEHICLE,193260
2,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,133780
3,Sedan,Station Wagon/Sport Utility Vehicle,123915
4,Station Wagon/Sport Utility Vehicle,Sedan,123812
...,...,...,...
6891,Pick-up Truck,UNKNOWN,1
6892,Pick-up Truck,UNK L,1
6893,Pick-up Truck,U HAU,1
6894,Pick-up Truck,Truck,1


## Question 9 - Among crashes where the contributing factor ```CONTRIBUTING FACTOR VEHICLE 1``` was alcohol involvement, what proportion resulted in a fatality?

In [35]:
df.rename(columns = {'VEHICLE TYPE CODE 1': 'vehicle_type_code_1', 
                     'VEHICLE TYPE CODE 2': 'vehicle_type_code_2', 
                     'CONTRIBUTING FACTOR VEHICLE 1': 'contributing_factor_vehicle_1'}, inplace = True)

In [36]:
df.rename(columns = {'NUMBER OF PERSONS KILLED': 'number_of_persons_killed'}, inplace  = True)

In [37]:
df.rename(columns = {'NUMBER OF PERSONS INJURED':'number_of_persons_injured'}, inplace = True)

In [38]:
df_alcohol_involvement = df.query('contributing_factor_vehicle_1 == "Alcohol Involvement" and number_of_persons_killed >=0 ')

In [39]:
df_total_fatalities = df.query('number_of_persons_killed >=1')

In [40]:
pd.crosstab(df_alcohol_involvement.contributing_factor_vehicle_1, df_alcohol_involvement.number_of_persons_killed, margins = True)

number_of_persons_killed,0.0,1.0,2.0,All
contributing_factor_vehicle_1,,,,
Alcohol Involvement,21555,99,2,21656
All,21555,99,2,21656


In [41]:
pd.crosstab(df_total_fatalities.contributing_factor_vehicle_1, df_total_fatalities.number_of_persons_killed, margins = True)

number_of_persons_killed,1.0,2.0,3.0,4.0,5.0,8.0,All
contributing_factor_vehicle_1,,,,,,,
Aggressive Driving/Road Rage,20,1,0,0,0,0,21
Alcohol Involvement,99,2,0,0,0,0,101
Animals Action,1,0,0,0,0,0,1
Backing Unsafely,40,0,0,0,0,0,40
Driver Inattention/Distraction,329,6,1,0,0,0,336
Driver Inexperience,50,1,1,1,0,0,53
Driverless/Runaway Vehicle,4,0,0,0,0,0,4
Drugs (Illegal),3,0,0,0,0,0,3
Drugs (illegal),8,0,0,0,0,0,8


In [42]:
print((101/21656)*100)

0.46638345031400075


## Question 10 - What proportion of crashes occur during the evening rush hour, defined as starting at 4 PM, and before 7 PM?

In [43]:
df['Hour'] = df['CRASH DATE_CRASH TIME'].dt.hour

In [44]:
df_4_to_7 = df.query('Hour >= 16 and Hour < 19')
df_4_to_7

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,number_of_persons_injured,...,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,vehicle_type_code_1,vehicle_type_code_2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,CRASH_YEAR,CRASH_MONTH,Hour
6,2021-12-14 17:05:00,NaN,NaN,40.709183,-73.956825,"(40.709183, -73.956825)",BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,0.0,...,NaN,4486555,Sedan,Tractor Truck Diesel,NaN,NaN,NaN,2021,12,17
11,2021-12-14 16:50:00,QUEENS,11413.0,40.675884,-73.755770,"(40.675884, -73.75577)",SPRINGFIELD BOULEVARD,EAST GATE PLAZA,NaN,0.0,...,NaN,4487127,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,2021,12,16
15,2021-12-14 17:58:00,BROOKLYN,11217.0,40.681580,-73.974630,"(40.68158, -73.97463)",NaN,NaN,480 DEAN STREET,0.0,...,NaN,4486604,Tanker,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,2021,12,17
24,2021-12-13 17:40:00,STATEN ISLAND,10301.0,40.631650,-74.087620,"(40.63165, -74.08762)",VICTORY BOULEVARD,WOODSTOCK AVENUE,NaN,1.0,...,NaN,4487001,Sedan,Sedan,NaN,NaN,NaN,2021,12,17
25,2021-12-14 17:31:00,BROOKLYN,11230.0,40.623104,-73.958090,"(40.623104, -73.95809)",EAST 18 STREET,AVENUE K,NaN,1.0,...,NaN,4486516,Sedan,NaN,NaN,NaN,NaN,2021,12,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018219,2023-07-17 16:39:00,BRONX,10451.0,40.816550,-73.919550,"(40.81655, -73.91955)",EAST 149 STREET,COURTLANDT AVENUE,NaN,0.0,...,NaN,4648282,Sedan,Box Truck,NaN,NaN,NaN,2023,7,16
2018223,2023-07-22 16:43:00,BROOKLYN,11225.0,40.655800,-73.962030,"(40.6558, -73.96203)",NaN,NaN,197 OCEAN AVENUE,0.0,...,NaN,4648050,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,2023,7,16
2018236,2023-07-22 16:15:00,NaN,NaN,NaN,NaN,NaN,PELHAM PARKWAY NORTH,STILLWELL AVENUE,NaN,1.0,...,NaN,4647987,Ambulance,Moped,NaN,NaN,NaN,2023,7,16
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN,2023,7,18


In [45]:
print(414023/2018245)

0.20514010935243243


## Question 11 - Among crashes involving motorcycles, what proportion resulted in injuries but no fatalities?

In [46]:
df.query('vehicle_type_code_1 == "Motorcycle" or vehicle_type_code_2 == "Motorcycle" and number_of_persons_injured >= 0 and number_of_persons_killed == 0')

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,number_of_persons_injured,...,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,vehicle_type_code_1,vehicle_type_code_2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,CRASH_YEAR,CRASH_MONTH,Hour
98,2022-03-21 12:05:00,MANHATTAN,10018.0,40.756320,-73.999275,"(40.75632, -73.999275)",NaN,NaN,515 WEST 36 STREET,1.0,...,NaN,4514237,Motorcycle,Sedan,NaN,NaN,NaN,2022,3,12
106,2022-03-26 14:05:00,BROOKLYN,11210.0,40.635063,-73.947880,"(40.635063, -73.94788)",NaN,NaN,2067 NOSTRAND AVENUE,1.0,...,NaN,4513874,Station Wagon/Sport Utility Vehicle,Motorcycle,NaN,NaN,NaN,2022,3,14
128,2022-03-23 09:00:00,NaN,NaN,40.676390,-73.971890,"(40.67639, -73.97189)",FLATBUSH AVENUE,NaN,NaN,1.0,...,NaN,4514160,Sedan,Motorcycle,NaN,NaN,NaN,2022,3,9
147,2022-03-25 17:55:00,NaN,NaN,40.683270,-73.950160,"(40.68327, -73.95016)",NOSTRAND AVENUE,NaN,NaN,0.0,...,NaN,4514241,Sedan,Motorcycle,NaN,NaN,NaN,2022,3,17
293,2021-08-29 00:12:00,NaN,NaN,40.611700,-74.139180,"(40.6117, -74.13918)",VICTORY BOULEVARD,HARVEY AVENUE,NaN,1.0,...,NaN,4456782,Motorcycle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,2021,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018022,2023-07-19 14:51:00,NaN,NaN,40.661690,-73.961430,"(40.66169, -73.96143)",FLATBUSH AVENUE,NaN,NaN,1.0,...,NaN,4648013,Station Wagon/Sport Utility Vehicle,Motorcycle,NaN,NaN,NaN,2023,7,14
2018061,2023-07-22 15:16:00,NaN,NaN,40.845990,-73.847890,"(40.84599, -73.84789)",WILLIAMSBRIDGE ROAD,SACKET AVENUE,NaN,0.0,...,NaN,4647486,Station Wagon/Sport Utility Vehicle,Motorcycle,NaN,NaN,NaN,2023,7,15
2018066,2023-07-22 11:30:00,BROOKLYN,11201.0,40.702408,-73.994415,"(40.702408, -73.994415)",NaN,NaN,12 EVERITT STREET,0.0,...,NaN,4648290,Sedan,Motorcycle,NaN,NaN,NaN,2023,7,11
2018068,2023-07-22 10:05:00,QUEENS,11369.0,40.763010,-73.875330,"(40.76301, -73.87533)",ASTORIA BOULEVARD,94 STREET,NaN,1.0,...,NaN,4648068,Motorcycle,Taxi,NaN,NaN,NaN,2023,7,10


In [47]:
df.query('vehicle_type_code_1 == "Motorcycle" or vehicle_type_code_2 == "Motorcycle"')

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,number_of_persons_injured,...,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,vehicle_type_code_1,vehicle_type_code_2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,CRASH_YEAR,CRASH_MONTH,Hour
98,2022-03-21 12:05:00,MANHATTAN,10018.0,40.756320,-73.999275,"(40.75632, -73.999275)",NaN,NaN,515 WEST 36 STREET,1.0,...,NaN,4514237,Motorcycle,Sedan,NaN,NaN,NaN,2022,3,12
106,2022-03-26 14:05:00,BROOKLYN,11210.0,40.635063,-73.947880,"(40.635063, -73.94788)",NaN,NaN,2067 NOSTRAND AVENUE,1.0,...,NaN,4513874,Station Wagon/Sport Utility Vehicle,Motorcycle,NaN,NaN,NaN,2022,3,14
128,2022-03-23 09:00:00,NaN,NaN,40.676390,-73.971890,"(40.67639, -73.97189)",FLATBUSH AVENUE,NaN,NaN,1.0,...,NaN,4514160,Sedan,Motorcycle,NaN,NaN,NaN,2022,3,9
147,2022-03-25 17:55:00,NaN,NaN,40.683270,-73.950160,"(40.68327, -73.95016)",NOSTRAND AVENUE,NaN,NaN,0.0,...,NaN,4514241,Sedan,Motorcycle,NaN,NaN,NaN,2022,3,17
293,2021-08-29 00:12:00,NaN,NaN,40.611700,-74.139180,"(40.6117, -74.13918)",VICTORY BOULEVARD,HARVEY AVENUE,NaN,1.0,...,NaN,4456782,Motorcycle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,2021,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018022,2023-07-19 14:51:00,NaN,NaN,40.661690,-73.961430,"(40.66169, -73.96143)",FLATBUSH AVENUE,NaN,NaN,1.0,...,NaN,4648013,Station Wagon/Sport Utility Vehicle,Motorcycle,NaN,NaN,NaN,2023,7,14
2018061,2023-07-22 15:16:00,NaN,NaN,40.845990,-73.847890,"(40.84599, -73.84789)",WILLIAMSBRIDGE ROAD,SACKET AVENUE,NaN,0.0,...,NaN,4647486,Station Wagon/Sport Utility Vehicle,Motorcycle,NaN,NaN,NaN,2023,7,15
2018066,2023-07-22 11:30:00,BROOKLYN,11201.0,40.702408,-73.994415,"(40.702408, -73.994415)",NaN,NaN,12 EVERITT STREET,0.0,...,NaN,4648290,Sedan,Motorcycle,NaN,NaN,NaN,2023,7,11
2018068,2023-07-22 10:05:00,QUEENS,11369.0,40.763010,-73.875330,"(40.76301, -73.87533)",ASTORIA BOULEVARD,94 STREET,NaN,1.0,...,NaN,4648068,Motorcycle,Taxi,NaN,NaN,NaN,2023,7,10


In [48]:
print(13761/13824)

0.9954427083333334


## Question 12 - How many crashes involved bicycles as one of the vehicles? (Consider ```VEHICLE TYPE CODE 1``` and ```VEHICLE TYPE CODE 2```)

In [49]:
df.query('vehicle_type_code_1 == "BICYCLE" or vehicle_type_code_2 == "BICYCLE"')

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,number_of_persons_injured,...,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,vehicle_type_code_1,vehicle_type_code_2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,CRASH_YEAR,CRASH_MONTH,Hour
218167,2020-10-06 16:43:00,QUEENS,11423.0,40.729210,-73.781166,"(40.72921, -73.781166)",188 STREET,UNION TURNPIKE,NaN,1.0,...,NaN,4355439,BICYCLE,NaN,NaN,NaN,NaN,2020,10,16
1077740,2016-07-07 08:07:00,QUEENS,11373.0,NaN,NaN,NaN,BROADWAY,BAXTER AVENUE,NaN,0.0,...,NaN,3485897,BICYCLE,PASSENGER VEHICLE,BICYCLE,NaN,NaN,2016,7,8
1092878,2016-06-17 16:06:00,BROOKLYN,11203.0,NaN,NaN,NaN,UTICA AVENUE,RUTLAND ROAD,NaN,1.0,...,NaN,3470666,BICYCLE,BICYCLE,NaN,NaN,NaN,2016,6,16
1093258,2016-06-18 03:40:00,QUEENS,11105.0,40.768888,-73.906908,"(40.7688877, -73.9069078)",SOUND STREET,ASTORIA BLVD NORTH,,0.0,...,NaN,3463912,BICYCLE,PASSENGER VEHICLE,NaN,NaN,NaN,2016,6,3
1144089,2016-04-05 20:27:00,NaN,NaN,NaN,NaN,NaN,FLATBUSH AVENUE,LINCOLN ROAD,,0.0,...,NaN,3417759,PASSENGER VEHICLE,BICYCLE,NaN,NaN,NaN,2016,4,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1924860,2012-07-09 17:37:00,BROOKLYN,11222,40.720601,-73.954754,"(40.7206006, -73.9547539)",BEDFORD AVENUE,NORTH 12 STREET,NaN,1.0,...,NaN,198192,TAXI,BICYCLE,NaN,NaN,NaN,2012,7,17
1924881,2012-07-02 09:46:00,MANHATTAN,10002,40.717724,-73.985765,"(40.7177239, -73.9857652)",DELANCEY STREET,CLINTON STREET,NaN,1.0,...,NaN,12187,PICK-UP TRUCK,BICYCLE,NaN,NaN,NaN,2012,7,9
1924949,2012-07-07 18:40:00,NaN,NaN,40.867335,-73.822707,"(40.8673349, -73.8227066)",NaN,NaN,NaN,1.0,...,NaN,2912116,PASSENGER VEHICLE,BICYCLE,NaN,NaN,NaN,2012,7,18
1924950,2012-07-06 13:33:00,BROOKLYN,11209,40.625780,-74.024154,"(40.6257805, -74.0241544)",5 AVENUE,80 STREET,NaN,1.0,...,NaN,140835,PASSENGER VEHICLE,BICYCLE,NaN,NaN,NaN,2012,7,13


# MAP of crashes with fatalities more than 2 in New York City.

In [1]:
#!pip3 install folium==0.5.0
import folium

In [59]:
df = df.dropna(subset = ['LATITUDE','LONGITUDE'])

In [64]:
df_fatalities = df.query('number_of_persons_killed >= 2')

In [68]:
latitude = 40.730610
longitude = -73.935242
New_york_city_map = folium.Map(location=[latitude, longitude], zoom_start=12)

crashes = folium.map.FeatureGroup()

for lat, lng, in zip(df_fatalities.LATITUDE, df_fatalities.LONGITUDE):
    crashes.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

latitudes = list(df_fatalities.LATITUDE)
longitudes = list(df_fatalities.LONGITUDE)
labels = list(map(str, df_fatalities.number_of_persons_killed))

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(New_york_city_map)

New_york_city_map.add_child(crashes)

New_york_city_map

# Conclusion

In this notebook a few code snippets of how to analyse the data and what could be some key feature to use to determine reasons for crash have been presented.


|Created By|Contact|
|----------|-------|
|Deepak Reddy|deepak.reddy0106@gmail.com|